In [7]:
%matplotlib widget

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import seaborn as sns
rocket = sns.color_palette("rocket", as_cmap=True)

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff()

from tqdm import tqdm
from toric import *

In [9]:
L = 50 # Lattice size
p_error = 0.05 # Error probability per spin
c = 10 # "Field velocity" - number of field updates per cycle
η = 0.5 # Smoothing paramter for Jacobi method
T = 10 # Epochs

In [10]:
def decoder_2D(state: State, T: int, c: int, η: float) -> np.ndarray:
    """
    Run a 2D decoder on a state for T epochs.

    Parameters:
    state (State): The state to decode.
    T (int): Number of epochs to run.
    c (int): Field velocity.
    η (float): Smoothing parameter.

    Returns:
    np.ndarray: T x L x L array representing the anyon position history.
    """
    q_history = []
    for _ in tqdm(range(T)):
        for _ in range(c):
            update_field(state, η)
        update_anyon(state)
        q_history.append(state.q.copy())
    return np.array(q_history)

In [11]:
def plot_evolution(q_history: np.ndarray, trail: bool) -> animation.FuncAnimation:
    """
    Plot the evolution of the anyon position history.

    Parameters:
    q_history (np.ndarray): T x L x L array representing the anyon position history.
    trail (bool): Whether to display a trail behind the anyon as it moves.

    Returns:
    animation.FuncAnimation: Animation of the anyon evolution.
    """

    fig, ax = plt.subplots()
    mat = ax.matshow(q_history[0,:,:], cmap = rocket)
    fig.colorbar(mat)

    def update(i):
        if trail and i > 0:
            previous = q_history[i-1::-1,:,:]
            weights = 2 ** np.arange(-1, -i-1, -1, dtype = np.float32)
            history = np.tensordot(previous, weights, axes = (0, 0))
            mat.set_data(np.maximum(q_history[i,:,:], history))
        else:
            mat.set_data(q_history[i,:,:])
        return mat
    
    return animation.FuncAnimation(fig = fig, func = update, frames = T, interval = 250)

In [12]:
q_history = decoder_2D(init_state(L, p_error), T, c, η)

100%|██████████| 10/10 [00:00<00:00, 143.49it/s]


In [14]:
plot_evolution(q_history, trail = False)